In [2]:
import os
from os.path import join
import pickle as pkl
import time
from easydict import EasyDict as edict
import pandas as pd
import h5py
import numpy as np
from neuro_data_analysis.mat_data_translate_lib import h5_to_dict_simplify, print_hdf5_info

mat_root = r"S:\Data-Ephys-MAT"
pkl_root = r"S:\Data-Ephys-PKL"
exp_record_pathdict = {"Alfa": r"S:\Exp_Record_Alfa.xlsx", 
                       "Beto": r"S:\ExpSpecTable_Augment.xlsx",
                       "Caos": r"S:\Exp_Record_Caos.xlsx",
                       "Diablito": r"S:\Exp_Record_Diablito.xlsx"}
for Animal, path in exp_record_pathdict.items():
    exp_record = pd.read_excel(path)
    exp_record.to_csv(path.replace(".xlsx", ".csv"), index=False)

In [3]:
ExpRecord_CD = pd.concat([pd.read_excel(exp_record_pathdict[Animal]) for Animal in ("Caos", "Diablito")])
exp_mask = ExpRecord_CD.Exp_collection.str.contains('BigGAN_FC6|BigGAN_Hessian', na=False) & ~ ExpRecord_CD.Expi.isna()
ExpRecord_CD.loc[exp_mask, :]

,ChanQual,Exp_collection,Expi,WebCam,comments,ephysFN,expControlFN,pref_chan,pref_unit,stim_size,stimuli
11,NaN,BigGAN_FC6,1.0,NaN,002 generate biggan\n 91 (0 0) 2 2 cmaes SU 1/...,Caos-12022024-002,241202_143902_Caos_generate_BigGAN,91.0,NaN,NaN,N:\Stimuli\Evolutions\2024\2024-12-02-Ev-Caos-...
12,NaN,BigGAN_Hessian,1.0,NaN,"Manifold with biggan\n 003 at 309 PM, manifold...",Caos-12022024-003,241202_150857_Caos_selectivity_basic,91.0,NaN,NaN,N:\Stimuli\Evolutions\2024\2024-12-02-Ev-Caos-...
14,NaN,BigGAN_FC6,2.0,NaN,"Let's try chan 94, with hash\n 005 biggan\n 94...",Caos-12022024-005,241202_153847_Caos_generate_BigGAN,94.0,NaN,NaN,N:\Stimuli\Evolutions\2024\2024-12-02-Ev-Caos-...
15,NaN,BigGAN_Hessian,2.0,NaN,"006 manifold\n alas, it's all dogs\n 006 at 3...",Caos-12022024-006,241202_155547_Caos_selectivity_basic,94.0,NaN,NaN,N:\Stimuli\Evolutions\2024\2024-12-02-Ev-Caos-...
17,NaN,BigGAN_FC6,3.0,NaN,002 biggan\nstarted at 218 PM\n71 (0 0) 2 2 CM...,Caos-12042024-002,241204_141711_Caos_generate_BigGAN,71.0,NaN,2.0,N:\Stimuli\Evolutions\2024\2024-12-04-Ev-Caos-...
18,NaN,BigGAN_Hessian,3.0,NaN,"003 at 239 PM\nmanifold\n(0,0) 2\nhas had arou...",Caos-12042024-003,241204_143902_Caos_selectivity_basic,71.0,NaN,2.0,N:\Stimuli\Evolutions\2024\2024-12-04-Ev-Caos-...
21,NaN,BigGAN_FC6,4.0,NaN,Will try another biggan\nch 72 ( 0 0) 2 1 CMAE...,Caos-12042024-006,241204_151045_Caos_generate_BigGAN,72.0,NaN,NaN,N:\Stimuli\Evolutions\2024\2024-12-04-Ev-Caos-...
22,NaN,BigGAN_Hessian,4.0,NaN,Running manifold\n007 at 3:30 PM\nch 72 ( 0 0)...,Caos-12042024-007,241204_152944_Caos_selectivity_basic,72.0,NaN,NaN,N:\Stimuli\Evolutions\2024\2024-12-04-Ev-Caos-...
25,NaN,BigGAN_FC6,5.0,NaN,Trying biggan again with SU in 92\n92 (0 0) 2 ...,Caos-12092024-003,241209_142223_Caos_generate_BigGAN,92.0,NaN,2.0,N:\Stimuli\Evolutions\2024\2024-12-09-Ev-Caos-...
26,NaN,BigGAN_Hessian,5.0,NaN,Manifold selectivity basic started at 2.48PM -...,Caos-12092024-004,241209_144750_Caos_selectivity_basic,92.0,NaN,2.0,N:\Stimuli\Evolutions\2024\2024-12-09-Ev-Caos-...


In [4]:
mat_root = r"S:\Data-Ephys-MAT"
pkl_root = r"S:\Data-Ephys-PKL"
# ephys_name = "Diablito-12122024-006"
T0 = time.time()
for ephys_name in ExpRecord_CD.loc[exp_mask, :].ephysFN.values:
    t0 = time.time()
    mat_file = join(mat_root, f"{ephys_name}_formatted.mat")
    data = h5py.File(mat_file, "r")
    print(f"Loading {ephys_name} mat file")
    t1 = time.time()
    print(f"Time taken: {t1 - t0:.2f} seconds for loading mat file")
    # fast selective loading
    meta_dict = h5_to_dict_simplify(data['meta'], data)
    # turn rasters into a numpy array, reduce precision to 32-bit float
    rasters = np.array(data['rasters']).astype(np.float32)
    # lfps = np.array(data['lfps']).astype(np.float32)
    Trial_dict = edict()
    for key in data['Trials'].keys():
        if key in ["B", "MLConfig"]:
            continue
        print(f"Loading {key}", end="\t")
        Trial_dict[key] = h5_to_dict_simplify(data['Trials'][key], data)
        print("done")
    # TrialRecord_dict = h5_to_dict_simplify(data['Trials/TrialRecord'], data)
    t2 = time.time()
    print(f"Time taken: {t2 - t1:.2f} seconds for loading trials")

    pkl.dump({"Trials": Trial_dict, "meta": meta_dict, "rasters": rasters, }, open(join(pkl_root, f"{ephys_name}.pkl"), "wb"))
    t3 = time.time()
    print(f"Time taken: {t3 - t2:.2f} seconds for dumping pkl file [Total: {t3 - T0:.2f} seconds]")


Loading Caos-12022024-002 mat file
Time taken: 0.00 seconds for loading mat file
Loading TrialRecord	Warning: No MATLAB class specified for <HDF5 dataset "Angle": shape (5, 1), type "|O"> at path /Trials/TrialRecord/CurrentConditionStimulusInfo/Angle object
done
Loading XY	done
Loading block	done
Loading event03	done
Loading event10	done
Loading eventMarkers	done
Loading eyePupil	done
Loading eyeXY	done
Loading imageInTrial	done
Loading imageName	done
Loading imageOFFtime	done
Loading imageONtime	done
Loading trialNum	done
Loading trialStart	done
Loading width	done
Loading words	done
Time taken: 20.57 seconds for loading trials
Time taken: 1.96 seconds for dumping pkl file [Total: 22.53 seconds]
Loading Caos-12022024-003 mat file
Time taken: 0.26 seconds for loading mat file
Loading TrialRecord	Warning: No MATLAB class specified for <HDF5 dataset "Angle": shape (6, 1), type "|O"> at path /Trials/TrialRecord/CurrentConditionStimulusInfo/Angle object
done
Loading XY	done
Loading block	do